In this notebook, we will import a couple of `.csv` files obtained from the National Household Income and Expenditure Survey (2024) performed in Mexico. Here's a link to the documentation:

https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/889463924494.pdf 

In order to develop our housing well-being classifier, we first need to import some basic libraries.


In [4]:
# Basic library importation
import pandas as pd
import json
import warnings
import numpy as np
warnings.filterwarnings('ignore')

# Show all columns
pd.set_option('display.max_columns', None)

Now we are going to import some .csv files from the microdata section of the INEGI (Mexico) website. In this project, five files were selected corresponding to the following tables:
* HOGARES : Characteristics of the households in which the members of the household live
* INGRESOS: Income and financial and capital gains of household members
* VIVIENDAS: Characteristics of the dwellings occupied by household members
* POBLACION: Sociodemographic characteristics of household members
* GASTOS_HOGAR: Household expenditure

In [5]:
# Data loading
HOGARES = pd.read_csv('hogares.csv') 
INGRESOS = pd.read_csv('ingresos.csv') 
VIVIENDAS = pd.read_csv('viviendas.csv') 
POBLACION = pd.read_csv('poblacion.csv') 
GASTOS_HOGAR = pd.read_csv('gastoshogar.csv') 

In [6]:
# Let's verify hogares was loaded
HOGARES.head()

,folioviv,foliohog,huespedes,huesp_come,num_trab_d,trab_come,acc_alim1,acc_alim2,acc_alim3,acc_alim4,acc_alim5,acc_alim6,acc_alim7,acc_alim8,acc_alim9,acc_alim10,acc_alim11,acc_alim12,acc_alim13,acc_alim14,acc_alim15,acc_alim16,alim17_1,alim17_2,alim17_3,alim17_4,alim17_5,alim17_6,alim17_7,alim17_8,alim17_9,alim17_10,alim17_11,alim17_12,acc_alim18,telefono,celular,conex_inte,tv_paga,peliculas,num_auto,anio_auto,num_van,anio_van,num_pick,anio_pick,num_moto,anio_moto,num_bici,anio_bici,num_trici,anio_trici,num_carre,anio_carre,num_canoa,anio_canoa,num_otro,anio_otro,num_ester,anio_ester,num_radio,anio_radio,num_tva,anio_tva,num_tvd,anio_tvd,num_dvd,anio_dvd,num_licua,anio_licua,num_tosta,anio_tosta,num_micro,anio_micro,num_refri,anio_refri,num_estuf,anio_estuf,num_lavad,anio_lavad,num_planc,anio_planc,num_maqui,anio_maqui,num_venti,anio_venti,num_aspir,anio_aspir,num_compu,anio_compu,num_lap,anio_lap,num_table,anio_table,num_impre,anio_impre,num_juego,anio_juego,tsalud1_h,tsalud1_m,camb_clim,f_sequia,f_inunda,f_helada,f_incendio,f_huracan,f_desliza,f_otro,af_viv,af_empleo,af_negocio,af_cultivo,af_trabajo,af_salud,af_otro,habito_1,habito_2,habito_3,habito_4,habito_5,habito_6,consumo,nr_viv,tarjeta,pagotarjet,regalotar,regalodado,autocons,regalos,remunera,transferen,parto_g,negcua,est_alim,est_trans,bene_licon,cond_licon,lts_licon,otros_lts,diconsa,frec_dicon,cond_dicon,pago_dicon,otro_pago,entidad,est_dis,upm,factor
0,100001901,1,0,,0,,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0,7,3,7,0,0,7,7,3,7,1,2,1,1,2,2,1,22.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,19.0,1,19.0,1,19.0,0,NaN,1,22.0,1,19.0,1,19.0,1,21.0,1,21.0,0,NaN,1,23.0,0,NaN,0,NaN,1,23.0,0,NaN,0,NaN,0,NaN,0,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,2.0,NaN,2,2,2,2,2,1,1,2,2,2,5100,0,2,NaN,NaN,,2,NaN,NaN,NaN,,1,1,1,207
1,100001902,1,0,,0,,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,3,3,7,4,7,1,7,7,7,7,7,1,1,1,1,1,1,1,24.0,0,NaN,0,NaN,1,24.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,22.0,0,NaN,1,24.0,1,14.0,1,19.0,0,NaN,0,NaN,1,12.0,1,13.0,1,16.0,1,10.0,0,NaN,0,NaN,0,NaN,0,NaN,1,21.0,0,NaN,1,17.0,0,NaN,1,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,5.0,NaN,1.0,NaN,1,2,2,2,2,1,1,2,2,2,12000,0,2,NaN,NaN,,1,3.0,NaN,NaN,,1,1,1,207
2,100001904,1,0,,0,,1,2,1,1,2,2,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,3,7,7,7,7,0,4,6,7,7,7,1,1,1,1,1,1,1,20.0,1,18.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,2,18.0,0,NaN,1,22.0,0,NaN,1,24.0,1,21.0,1,21.0,1,21.0,1,16.0,1,16,1,24.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,5.0,NaN,2.0,NaN,2,2,2,2,2,1,1,2,2,2,4350,0,2,NaN,NaN,,2,NaN,NaN,NaN,,1,1,1,207
3,100001905,1,0,,0,,1,2,2,2,2,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,0,5,5,4,7,0,5,7,7,5,7,1,1,1,1,1,2,1,14.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,9.0,0,NaN,0,NaN,2,14.0,0,NaN,1,9.0,0,NaN,1,16.0,1,8.0,1,12.0,1,9.0,1,23.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,19.0,0,NaN,0,NaN,0,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,5.0,NaN,1.0,NaN,1,1,2,2,2,2,1,1,2,2,8000,0,2,NaN,NaN,,9,NaN,NaN,NaN,,1,1,1,207
4,100002501,1,0,,0,,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,3,7,4,3,3,2,1,7,3,7,2,1,1,1,1,1,1,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,16.0,0,NaN,0,NaN,2,23.0,0,NaN,1,22.0,0,NaN,0,NaN,1,23.0,1,16.0,1,20.0,1,22.0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,2,23.0,0,NaN,0,NaN,0,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,NaN,1,2,2,2,2,2,1,2,2,2,12000,2000,2,NaN,NaN,,2,NaN,NaN,NaN,,1,2,2,196


# Table 1 : Hogares
The variables from the `hogares` table that will be considered for future analyses are as follows:

* __folioviv__ : housing identifier
* __foliohog__ : household identifier. 1 identifies the main household and 2 the additional households.
* acc_alim3 : guests have little food variety. 1 = yes, 2 = no.
* alim17_1 : number of days they ate cereals during the last week.
* alim17_5 : number of days they ate meat during the last week.
* alim17_6 : number of days they ate eggs during the last week.
* alim17_7 : number of days they ate fish during the last week.
* telefono : landline phone. 1 = yes, 2 = no.
* celular : mobile phone. 1 = yes, 2 = no.
* conex_inte : has internet connection. 1 = yes, 2 = no.
* num_auto : number of cars in the household
* num_tvd : number of digital televisions in the household
* num_compu : number of computers in the household
* num_lap : number of laptops acquired
* num_table : number of tablets acquired
* tarjeta : someone in the household has a credit card. 1 = yes, 2 = no
* pagotarjet : made payments with cards. 1 = yes, 2 = no
* necgua : does any member have a business? 1 = yes, 2 = no
* est_alim : estimated monthly spending on food
* est_trans : estimated monthly spending on transportation
* entidad : federal entity number

We will change some column names as well as transform some features to enhance further analysis. 

In [7]:
# Variable definition
feature_cols = ['folioviv', 'foliohog', 'acc_alim3', 'alim17_1','alim17_5','alim17_6',
                'alim17_7','telefono', 'celular', 'conex_inte', 'num_auto', 'num_tvd',
               'num_compu', 'num_lap', 'num_table', 'tarjeta','pagotarjet','negcua',
               'est_alim','est_trans', 'entidad']

# New dataframe creation
hogares = HOGARES[feature_cols]

# Change some variable names
hogares.rename(columns={'acc_alim3':'poca_variedad_alimentos','alim17_1':'alimentacion_cereales',
               'alim17_5':'alimentacion_carne','alim17_6':'alimentacion_huevo',
               'alim17_7':'alimentacion_pescado'}, inplace=True)

# Binaries variable transformation where 'yes' wil beconme 1 and 'no' will become 0
binaries = ['poca_variedad_alimentos','telefono', 'celular', 'conex_inte','tarjeta','pagotarjet','negcua']

# Change all rows in binarie columns. 1= will become 'yes' and 0 will become 'no'
for col in binaries:
    hogares[col] = hogares[col].replace({1:1,2:0})

# See the result
hogares

,folioviv,foliohog,poca_variedad_alimentos,alimentacion_cereales,alimentacion_carne,alimentacion_huevo,alimentacion_pescado,telefono,celular,conex_inte,num_auto,num_tvd,num_compu,num_lap,num_table,tarjeta,pagotarjet,negcua,est_alim,est_trans,entidad
0,100001901,1,0,7,3,7,0,0,1,1,1,1,0,1,0,0,0,0,5100,0,1
1,100001902,1,0,7,4,7,1,1,1,1,1,1,0,1,0,1,0,0,12000,0,1
2,100001904,1,1,7,7,7,0,1,1,1,1,2,0,0,0,0,0,0,4350,0,1
3,100001905,1,0,7,4,7,0,1,1,1,1,2,0,0,1,1,1,0,8000,0,1
4,100002501,1,0,7,3,3,2,1,1,1,0,2,0,0,2,1,0,0,12000,2000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91409,3260593814,1,0,7,1,2,0,0,1,1,0,1,0,0,0,0,0,0,6000,0,32
91410,3260593815,1,0,7,4,6,1,0,1,1,1,1,0,1,1,1,0,0,6000,0,32
91411,3260593816,1,0,7,3,7,0,0,1,1,0,1,0,0,0,0,0,0,3800,420,32
91412,3260593817,1,0,7,1,7,0,1,1,1,0,1,0,1,0,0,0,0,2800,0,32


## Table 2: Ingresos

The variables from the `ingresos` table that will be considered for future analyses are as follows:

* __folioviv__ : housing identifier
* __foliohog__ : household identifier. 1 identifies the main household and 2 the additional households.
* numren : consecutive number in the record of household members. This means that a single `folioviv` can have multiple `numren`, indicating that several people live there.
* clave : refers to the source of income (salary, piecework, commissions, overtime, etc.). This means that a single `folioviv` can have multiple `clave`.
* ing_1 : income from last month
* ing_2 : income from the second-to-last month
* ing_3 : income from the third-to-last month
* ing_4 : income from the fourth-to-last month
* ing_5 : income from the fifth-to-last month
* ing_6 : income from the sixth-to-last month
* entidad : federal entity number

As before, we will perform some data modification and transformation. In this table, we will calculate the average monthly income per person livind in a specific household. Then, we will group this average income per person into an average income per household. 

In [8]:
#Let's verify ingresos was loaded
INGRESOS.head()

,folioviv,foliohog,numren,clave,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,ing_1,ing_2,ing_3,ing_4,ing_5,ing_6,ing_tri,entidad,est_dis,upm,factor
0,100001901,1,1,P001,10,9,8,7,6,5,16680,16680,16680,16680,16680,16680,48952.17,1,1,1,207
1,100001901,1,1,P004,10,9,8,7,6,5,6000,6000,6000,6000,6000,6000,17608.69,1,1,1,207
2,100001901,1,1,P006,10,9,8,7,6,5,1500,1500,1500,1500,1500,1500,4402.17,1,1,1,207
3,100001901,1,1,P008,10,9,8,7,6,5,18000,,,,,,4402.17,1,1,1,207
4,100001901,1,1,P009,10,9,8,7,6,5,14000,,,,,,3423.91,1,1,1,207


In [9]:
# Variable definition
feature_cols2 = ['folioviv','foliohog','numren', 'clave','ing_1','ing_2','ing_3',
                'ing_4','ing_5','ing_6','entidad']

# New dataframe creation with last variables
ingresos = INGRESOS[feature_cols2]

# Change datatype to perform calculations
cols_ingresos = ['ing_1','ing_2','ing_3','ing_4','ing_5','ing_6']
ingresos[cols_ingresos] = ingresos[cols_ingresos].replace(r'^\s*$', 0, regex=True) # regular expression that uses cells that only have spaces or are empty

# Column data transformation to floating data
ingresos[cols_ingresos] = ingresos[cols_ingresos].astype(float)

# Calculate average monthly earnings
ingresos['ingreso_promedio'] = ingresos[['ing_1','ing_2','ing_3','ing_4','ing_5','ing_6']].mean(axis=1)

# Now let's group earnings by housing or folioviv
ingresos_vivienda = ingresos.groupby(['folioviv','foliohog','entidad'],as_index=False).agg(ingreso_promedio=('ingreso_promedio','sum'))

# Round decimals
ingresos_vivienda['ingreso_promedio'] = ingresos_vivienda['ingreso_promedio'].round(2)
ingresos_vivienda

,folioviv,foliohog,entidad,ingreso_promedio
0,100001901,1,1,40546.67
1,100001902,1,1,32666.67
2,100001904,1,1,14083.33
3,100001905,1,1,30700.00
4,100002501,1,1,44288.00
...,...,...,...,...
91347,3260593814,1,32,12373.33
91348,3260593815,1,32,32000.00
91349,3260593816,1,32,15991.67
91350,3260593817,1,32,1133.33


# Table 3 :  Viviendas

The variables from the `viviendas` table that will be considered for future analyses are as follows:

* __folioviv__ : housing identifier
* tipo_viv : type of housing. 1 = single house on the lot, 2 = house sharing the lot with others, 3 = duplex house, 4 = apartment in a building, 5 = housing in a neighborhood, 6 = housing on rooftop, 7 = premises not built for habitation.
* mat_pared : wall material. 1 = waste material, 2 = cardboard sheet, 3 = asbestos or metal sheet, 4 = cane, bamboo, or palm, 5 = mud or wattle, 6 = wood, 7 = adobe, 8 = brick, block, cement, or concrete.
* mat_pisos : floor material. 1 = dirt, 2 = cement or concrete, 3 = wood, mosaic, or other covering.
* num_cuarto : total number of rooms.
* ab_agua : water supply. 1 = public water service, 2 = community well, 3 = private well, 4 = water truck, 5 = from another house, 6 = rainwater, 7 = other source.
* disp_elect : electricity availability. 1 = public service, 2 = private plant, 3 = solar panel, 4 = other source, 5 = no electricity.
* renta : monthly rent amount
* pago_viv : monthly payment for housing
* est_socio : socioeconomic stratum; classification of housing in the country according to the socioeconomic characteristics of the inhabitants, as well as physical characteristics and equipment. 1 = Low, 2 = Lower-Middle, 3 = Upper-Middle, 4 = High

In this table, we will perform some data transformations as well as some data corrections. 


In [10]:
# Variable definition
feature_cols3 = ['folioviv','tipo_viv','mat_pared','mat_pisos','num_cuarto','ab_agua','disp_elect','est_socio']

# New dataframe creation
viviendas = VIVIENDAS[feature_cols3]


# Categorical column definition
categorical_cols = ['tipo_viv','mat_pared','mat_pisos','ab_agua','disp_elect']

# Categorial column transformation into strings
viviendas[categorical_cols] = viviendas[categorical_cols].astype(str)

# Verify unique values 
for col in categorical_cols:
    print(col, viviendas[col].unique())


# Correct values on tipo_viv variable
valores = ['1','2','3','4','5','6','7']
mask = viviendas['tipo_viv'].isin(valores)
viviendas = viviendas[mask]

# Correct values on mat_pisos variable
valores = ['1','2','3']
mask = viviendas['mat_pisos'].isin(valores)
viviendas = viviendas[mask]

# Correct values on ab_agua variable
valores = ['1.0','2.0','3.0','4.0','5.0','6.0']
mask = viviendas['ab_agua'].isin(valores)
viviendas = viviendas[mask]

tipo_viv ['7' '1' '4' '2' '3' '5' '6' '&']
mat_pared ['8' '1' '7' '6' '2' '5' '3' '4']
mat_pisos ['3' '2' '1' '&']
ab_agua ['1.0' '5.0' 'nan' '2.0' '3.0' '4.0' '7.0' '6.0']
disp_elect ['1' '5' '3' '4' '2']


In [11]:
# Let's check categorical columns are fixed
for col in categorical_cols:
    print(col, viviendas[col].unique())
    
viviendas

tipo_viv ['7' '1' '4' '2' '3' '5' '6']
mat_pared ['8' '1' '7' '6' '5' '3' '2' '4']
mat_pisos ['3' '2' '1']
ab_agua ['1.0' '5.0' '2.0' '3.0' '4.0' '6.0']
disp_elect ['1' '5' '3' '4' '2']


,folioviv,tipo_viv,mat_pared,mat_pisos,num_cuarto,ab_agua,disp_elect,est_socio
0,100001901,7,8,3,4,1.0,1,3
1,100001902,1,8,3,4,1.0,1,3
2,100001904,1,8,3,3,1.0,1,3
3,100001905,1,8,3,3,1.0,1,3
4,100002501,4,8,3,3,1.0,1,2
...,...,...,...,...,...,...,...,...
90319,3260593814,1,8,3,2,2.0,1,2
90320,3260593815,1,8,2,4,1.0,1,2
90321,3260593816,1,8,3,3,1.0,1,2
90322,3260593817,1,8,2,4,2.0,1,2


Now we will transform the `vivienda` df categorical variables into numerical variables using `OneHotEncoder`. One-Hot Encoding is a technique used to convert categorical variables into a numerical format that machine learning algorithms can understand. It's a fundamental part of the data preprocessing stage in many data science projects. In this case, we will transform some categorical features such as `tipo_viv`,`mat_pared`,`mat_pisos`,`ab_agua`, and `disp_elect`.

In [12]:
# Now we will apply onehotencoder from sklearn into categorical cols
from sklearn.preprocessing import OneHotEncoder

# Fill NaN values
viviendas[categorical_cols] = viviendas[categorical_cols].fillna('missing')

# Encoder creation
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Apply encoder
one_hot_encoded = encoder.fit_transform(viviendas[categorical_cols])

# Create dataframe with same index
variables_codificadas = pd.DataFrame(one_hot_encoded, 
                                    columns=encoder.get_feature_names_out(categorical_cols),
                                    index=viviendas.index)

# Merge dataframes and drop original columns
viviendas_onehot = pd.concat([viviendas, variables_codificadas], axis=1)
viviendas_onehot = viviendas_onehot.drop(categorical_cols, axis=1)
viviendas_onehot

,folioviv,num_cuarto,est_socio,tipo_viv_2,tipo_viv_3,tipo_viv_4,tipo_viv_5,tipo_viv_6,tipo_viv_7,mat_pared_2,mat_pared_3,mat_pared_4,mat_pared_5,mat_pared_6,mat_pared_7,mat_pared_8,mat_pisos_2,mat_pisos_3,ab_agua_2.0,ab_agua_3.0,ab_agua_4.0,ab_agua_5.0,ab_agua_6.0,disp_elect_2,disp_elect_3,disp_elect_4,disp_elect_5
0,100001901,4,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100001902,4,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100001904,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100001905,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100002501,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90319,3260593814,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90320,3260593815,4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90321,3260593816,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90322,3260593817,4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Table 4 : Poblacion

The variables from the `poblacion` table that will be considered for future analyses are as follows:

* __folioviv__ : housing identifier
* __foliohog__ : household identifier. 1 identifies the main household and 2 the additional households.
* __numren__ : consecutive number in the record of household members. This means that a single `folioviv` can have multiple `numren`, indicating that several people live there.
* parentesco (modified) : the person is a partner, relative, or other.
* sexo : 1 = male, 2 = female
* edad : age
* alfabetism : the person can read and write. 1 = yes, 2 = no
* disc_ver, disc_oir, disc_caminar, disc_usar_brazos, disc_aprender, * disc_recordar : disabilities in general. 1 = yes, 2 = no
* asis_esc : the person attends school. 1 = yes, 2 = no
* hor_1 : hours spent working per week
* hor_8 : hours dedicated to personal activities
* num_trabaj : number of jobs household members held during the past month. 1 = only 1, 2 = two or more
* entidad federativa : federal entity
* acceso_cols (created) : has access to healthcare. 1 = yes, 2 = no

In the original INEGI's microdata (see documentation), there's a couple of columns indicating the presence of a person with disability in the household. Instead of using all those columns, we will define the number of people with disability living in the current household. In addition, we will perform some data modifications and transformations such as deletion of duplicated and NaN values, among others.

Once performed all modifications, the table will be grouped by `folioviv`, which is the household identifier. 

In [13]:
# Variables definition
feature_cols4 = ['folioviv','foliohog','numren','parentesco','sexo','edad',
                 'hablaind', 'alfabetism','disc_ver', 'disc_oir','disc_brazo','disc_camin',
                'disc_apren','disc_vest','disc_habla','disc_acti','asis_esc','hor_1','hor_8',
                'num_trabaj','entidad']
poblacion = POBLACION [feature_cols4]

# Create single column to define disability. 1=yes, 0=no
disc = ['disc_ver','disc_oir','disc_brazo','disc_camin','disc_apren','disc_vest','disc_habla','disc_acti']

# Replace any value that is not 1,2,3,4 for NaN
poblacion[disc] = poblacion[disc].apply(lambda x: x.where(x.isin(['1','2','3','4'])))

# Convert to numeric data
poblacion[disc] = poblacion[disc].astype(float)

# Create binarie column for disability variable
poblacion['discapacidad'] = (poblacion[disc] == 1).any(axis=1).astype(int) #nos regresa un valor booleano True si alguna de las columnas presenta un 1. En caso de presentar un 2 o un NaN, se devuelve un False

# Drop original disability columns
poblacion = poblacion.drop(disc, axis=1)

# Now let's simplify parent column creating a dictionary
parent = {
    'Pareja': [101,201,202,203,204,205,413],
    'Familiar': [301,302,303,304,305,502,503,601,602,603,604,605,606,607,608,609,610,611,612,613,614],
    'Otro': [401,402,403,404,405,406,407,408,409,410,411,412,413,421,431,441,451,461]
}

# Apply function
def map_parentesco(codigo):
    for categoria, codigos in parent.items():
        if codigo in codigos:
            return categoria
    return 'Desconocido'

poblacion['parentesco_cat'] = poblacion['parentesco'].apply(map_parentesco)
poblacion = poblacion.drop('parentesco', axis=1)


# Check list of columns
cols_binarias = ['hablaind','alfabetism', 'asis_esc']
    
# Substitute NaN values for 0 
poblacion[cols_binarias] = poblacion[cols_binarias].fillna(0).astype(int)

# Check NaN percentaje
for col in cols_binarias:
    pct_nan = poblacion[col].isna().mean() * 100
    print(f"{col}: {pct_nan:.2f}% NaN")

# Substitute blank spaces in column hor_1 and hor_8
poblacion[['hor_1','hor_8']] = poblacion[['hor_1','hor_8']].replace(' ','0').astype(int)


# Substitute NaN and blank space values on num_trabaj
poblacion['num_trabaj'] = poblacion['num_trabaj'].fillna('0')
poblacion['num_trabaj'] = poblacion['num_trabaj'].replace(' ','0').astype(int)


# Calculate aggregate functions
cols_num = ['sexo', 'alfabetism', 'asis_esc', 'num_trabaj', 'discapacidad', 'edad']

for col in cols_num:
    poblacion[col] = pd.to_numeric(poblacion[col], errors='coerce')


# Aggregate dataframe
poblacion_agg = poblacion.groupby('folioviv').agg({
    'numren':'count', # Contar el numero de personas existentes
    'edad' : 'mean', #edad promedio
    'alfabetism' : 'mean', #porcentaje de alfabetizados
    'asis_esc':'count', #porcentaje de los que asisten a la escuela
    'num_trabaj':'count', # porcentaje de los que trabajan
    'discapacidad':'count' # porcentaje con discapacidad
}).reset_index()
poblacion_agg = poblacion_agg.round()
poblacion_agg

hablaind: 0.00% NaN
alfabetism: 0.00% NaN
asis_esc: 0.00% NaN


,folioviv,numren,edad,alfabetism,asis_esc,num_trabaj,discapacidad
0,100001901,4,16.0,1.0,4,4,4
1,100001902,4,34.0,1.0,4,4,4
2,100001904,2,44.0,1.0,2,2,2
3,100001905,4,28.0,1.0,4,4,4
4,100002501,4,19.0,1.0,4,4,4
...,...,...,...,...,...,...,...
90319,3260593814,3,37.0,1.0,3,3,3
90320,3260593815,5,16.0,1.0,5,5,5
90321,3260593816,5,29.0,1.0,5,5,5
90322,3260593817,2,54.0,1.0,2,2,2


Now we are going to consolidate our dataframe. To do this, we will use the merge() method and the parameters ‘inner’ and ‘folioviv’ as the key. Once the dataframe has been consolidated, we will check for NaN values and duplicate rows and columns.

In [14]:
# Transform data into int
viviendas_onehot['folioviv'] = viviendas_onehot['folioviv'].astype(int)

# Merge ingresos_vivienda with hogares
df = pd.merge(ingresos_vivienda, hogares, on='folioviv', how='inner')

# Merge last df with poblacion_agg
df = pd.merge(df, poblacion_agg, on='folioviv', how='inner')

# Merge last df with viviendas
df = pd.merge(df,viviendas, on='folioviv', how='inner')

# Verify NaN values
print('NaN values')
print(df.isna().sum(),'\n\n')

# Verify duplicate rows
print('Filas duplicadas')
print(df.duplicated().sum())

# Drop duplicate columns
df.drop(columns=['foliohog_y','entidad_y'],inplace=True)

# Rename columns
df.rename(columns={'foliohog_x' :'foliohog', 'entidad_x':'entidad'})

df

NaN values
folioviv                   0
foliohog_x                 0
entidad_x                  0
ingreso_promedio           0
foliohog_y                 0
poca_variedad_alimentos    0
alimentacion_cereales      0
alimentacion_carne         0
alimentacion_huevo         0
alimentacion_pescado       0
telefono                   0
celular                    0
conex_inte                 0
num_auto                   0
num_tvd                    0
num_compu                  0
num_lap                    0
num_table                  0
tarjeta                    0
pagotarjet                 0
negcua                     0
est_alim                   0
est_trans                  0
entidad_y                  0
numren                     0
edad                       0
alfabetism                 0
asis_esc                   0
num_trabaj                 0
discapacidad               0
tipo_viv                   0
mat_pared                  0
mat_pisos                  0
num_cuarto                 0
ab_

,folioviv,foliohog_x,entidad_x,ingreso_promedio,poca_variedad_alimentos,alimentacion_cereales,alimentacion_carne,alimentacion_huevo,alimentacion_pescado,telefono,celular,conex_inte,num_auto,num_tvd,num_compu,num_lap,num_table,tarjeta,pagotarjet,negcua,est_alim,est_trans,numren,edad,alfabetism,asis_esc,num_trabaj,discapacidad,tipo_viv,mat_pared,mat_pisos,num_cuarto,ab_agua,disp_elect,est_socio
0,100001901,1,1,40546.67,0,7,3,7,0,0,1,1,1,1,0,1,0,0,0,0,5100,0,4,16.0,1.0,4,4,4,7,8,3,4,1.0,1,3
1,100001902,1,1,32666.67,0,7,4,7,1,1,1,1,1,1,0,1,0,1,0,0,12000,0,4,34.0,1.0,4,4,4,1,8,3,4,1.0,1,3
2,100001904,1,1,14083.33,1,7,7,7,0,1,1,1,1,2,0,0,0,0,0,0,4350,0,2,44.0,1.0,2,2,2,1,8,3,3,1.0,1,3
3,100001905,1,1,30700.00,0,7,4,7,0,1,1,1,1,2,0,0,1,1,1,0,8000,0,4,28.0,1.0,4,4,4,1,8,3,3,1.0,1,3
4,100002501,1,1,44288.00,0,7,3,3,2,1,1,1,0,2,0,0,2,1,0,0,12000,2000,4,19.0,1.0,4,4,4,4,8,3,3,1.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88181,3260593814,1,32,12373.33,0,7,1,2,0,0,1,1,0,1,0,0,0,0,0,0,6000,0,3,37.0,1.0,3,3,3,1,8,3,2,2.0,1,2
88182,3260593815,1,32,32000.00,0,7,4,6,1,0,1,1,1,1,0,1,1,1,0,0,6000,0,5,16.0,1.0,5,5,5,1,8,2,4,1.0,1,2
88183,3260593816,1,32,15991.67,0,7,3,7,0,0,1,1,0,1,0,0,0,0,0,0,3800,420,5,29.0,1.0,5,5,5,1,8,3,3,1.0,1,2
88184,3260593817,1,32,1133.33,0,7,1,7,0,1,1,1,0,1,0,1,0,0,0,0,2800,0,2,54.0,1.0,2,2,2,1,8,2,4,2.0,1,2


The final selected variables are:
* __folioviv__ : housing identifier
* __foliohog__ : household identifier. 1 identifies the main household and 2 the additional households.
* entidad : federal entity number
* ingreso_promedio : monthly salary
* poca_variedad_alimentos : guests have little food variety. 1 = yes, 2 = no.
* alimentacion_cereales : number of days they ate cereals during the last week.
* alimentacion_carne : number of days they ate meat during the last week.
* alimentacion_huevo : number of days they ate eggs during the last week.
* alimentacion_pescado : number of days they ate fish during the last week.
* telefono : landline phone. 1 = yes, 2 = no.
* celular : mobile phone. 1 = yes, 2 = no.
* conex_inte : has internet connection. 1 = yes, 2 = no.
* num_auto : number of cars in the household
* num_tvd : number of digital televisions in the household
* num_compu : number of computers in the household
* num_lap : number of laptops acquired
* num_table : number of tablets acquired
* tarjeta : someone in the household has a credit card. 1 = yes, 2 = no
* pagotarjet : made payments with cards. 1 = yes, 2 = no
* necgua : does any member have a business? 1 = yes, 2 = no
* est_alim : estimated monthly spending on food
* est_trans : estimated monthly spending on transportation
* edad : average age in the household
* alfabetism : percentaje of persons in the household that can read and write.
* asis_esc : number of persons attending school. 1 = yes, 2 = no
* num_trabaj : number of persons in the household that are currently working. 1 = only 1, 2 = two or more
* discapacidad: number of persons in the household with disability p. 1 = yes, 2 = no
* tipo_viv : type of housing. 1 = single house on the lot, 2 = house sharing the lot with others, 3 = duplex house, 4 = apartment in a building, 5 = housing in a neighborhood, 6 = housing on rooftop, 7 = premises not built for habitation.
* mat_pared : wall material. 1 = waste material, 2 = cardboard sheet, 3 = asbestos or metal sheet, 4 = cane, bamboo, or palm, 5 = mud or wattle, 6 = wood, 7 = adobe, 8 = brick, block, cement, or concrete.
* mat_pisos : floor material. 1 = dirt, 2 = cement or concrete, 3 = wood, mosaic, or other covering.
* num_cuarto : total number of rooms.
* ab_agua : water supply. 1 = public water service, 2 = community well, 3 = private well, 4 = water truck, 5 = from another house, 6 = rainwater, 7 = other source.
* disp_elect : electricity availability. 1 = public service, 2 = private plant, 3 = solar panel, 4 = other source, 5 = no electricity.
* renta : monthly rent amount
* pago_viv : monthly payment for housing
* est_socio : socioeconomic stratum; classification of housing in the country according to the socioeconomic characteristics of the inhabitants, as well as physical characteristics and equipment. 1 = Low, 2 = Lower-Middle, 3 = Upper-Middle, 4 = High


Now that we have a single consolidated dataframe with all our variables of interest, we are going to export our dataframe to a .csv file, which will be the basis for working on the next steps of this project.